In [1]:
import duckdb
import pandas as pd

%load_ext sql
conn = duckdb.connect()
%sql duckdb:///:memory:
%sql conn --alias duckdb

%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

Connecting to 'duckdb:///:memory:'

In [2]:
%%sql
CREATE TABLE CollectedResults (
   RACE                 TEXT
  ,YEAR                 INTEGER
  ,STAGE_ID             TEXT
  ,STAGE_DATE           TEXT
  ,STAGE_NAME           TEXT
  ,STAGE_PROFILE        TEXT
  ,STAGE_TYPE           TEXT
  ,RANK                 TEXT
  ,RIDER                TEXT
  ,TEAM                 TEXT
  ,UCI_POINTS           DOUBLE
  ,PCS_POINTS           DOUBLE
  ,UPDATE_TIMESTAMP     DATETIME
);

,Count


In [7]:
%%sql
INSERT INTO CollectedResults 
SELECT 
   RaceName
  ,RaceYear
  ,StageNumber
  ,StageDate
  ,StageName
  ,'Unknown'
  ,StageType
  ,RiderRank
  ,RiderName
  ,TeamName
  ,TRY_CAST(UCIPoints AS DOUBLE)
  ,TRY_CAST(PCSPoints AS DOUBLE)
  ,UpdateTimeStamp
FROM read_csv_auto("../data/results_races/*.csv", union_by_name = True);

,Count
0,31866


In [9]:
%%sql
SELECT
     RaceName
    ,RaceYear
    ,StageName
    ,CASE 
        WHEN StageType = 'GC' THEN 'GC'
        WHEN StageType = 'TTT' THEN 'No points'
        ELSE 'Stage'
     END AS PointsGiven
    ,StageDate AS RaceDate
    ,MAX(StageDate) OVER(PARTITION BY RaceName) AS RaceMaxDate
    ,RiderRank
    ,TRY_CAST(RiderRank AS INTEGER) AS RiderRankInt
    ,REGEXP_REPLACE(REGEXP_REPLACE(RiderName, TeamName, ''), 'fav_gc', '') AS RiderNameClean
    ,TRY_CAST(UCIPoints AS INTEGER) AS UCIPoints
    ,TRY_CAST(PCSPoints AS INTEGER) AS PCSPoints
    ,UpdateTimeStamp
FROM "../data/results_races/*.csv";

,RaceName,RaceYear,StageName,PointsGiven,RaceDate,RaceMaxDate,RiderRank,RiderRankInt,RiderNameClean,UCIPoints,PCSPoints,UpdateTimeStamp
0,great-ocean-road-race,2025,Result,Stage,None,None,60,60,Watson Samuel,1,5,2025-08-24 22:31:49
1,great-ocean-road-race,2025,Result,Stage,None,None,DNF,<NA>,Le Huitouze Eddy,<NA>,<NA>,2025-08-24 22:31:49
2,great-ocean-road-race,2025,Result,Stage,None,None,70,70,Maas Jan,<NA>,5,2025-08-24 22:31:49
3,great-ocean-road-race,2025,Result,Stage,None,None,69,69,Herrada Jesús,<NA>,5,2025-08-24 22:31:49
4,great-ocean-road-race,2025,Result,Stage,None,None,68,68,MacKellar Alastair,<NA>,5,2025-08-24 22:31:49
...,...,...,...,...,...,...,...,...,...,...,...,...
1738,tour-down-under,2025,Stage 3,Stage,23/01,26/01,46,46,Pescador Diego,<NA>,<NA>,2025-08-24 22:30:54
1739,tour-down-under,2025,Stage 3,Stage,23/01,26/01,47,47,Izagirre Ion,<NA>,<NA>,2025-08-24 22:30:54
1740,tour-down-under,2025,Stage 3,Stage,23/01,26/01,48,48,Mulubrhan Henok,<NA>,<NA>,2025-08-24 22:30:54
1741,tour-down-under,2025,Stage 3,Stage,23/01,26/01,49,49,Zwiehoff Ben,<NA>,<NA>,2025-08-24 22:30:54


In [5]:
%%sql
SELECT * FROM "../data/results/results_2025_full.csv";

,RACE,YEAR,STAGE_ID,STAGE_DATE,STAGE_NAME,STAGE_PROFILE,STAGE_TYPE,RANK,RIDER,TEAM,UCI_POINTS,PCS_POINTS,UPDATE_TIMESTAMP
0,tour-down-under,2025,stage-1,2025-01-21,Stage 1,p2,Normal,1,Welsford Sam,Red Bull - BORA - hansgrohe,0.0,0.0,2025-02-17 17:22:30
1,tour-down-under,2025,stage-1,2025-01-21,Stage 1,p2,Normal,2,Brennan Matthew,Team Visma | Lease a Bike,0.0,0.0,2025-02-17 17:22:30
2,tour-down-under,2025,stage-1,2025-01-21,Stage 1,p2,Normal,3,Walls Matthew,Groupama - FDJ,0.0,0.0,2025-02-17 17:22:30
3,tour-down-under,2025,stage-1,2025-01-21,Stage 1,p2,Normal,4,Teutenberg Tim Torn,Lidl - Trek,0.0,0.0,2025-02-17 17:22:30
4,tour-down-under,2025,stage-1,2025-01-21,Stage 1,p2,Normal,5,Swift Ben,INEOS Grenadiers,0.0,0.0,2025-02-17 17:22:30
...,...,...,...,...,...,...,...,...,...,...,...,...,...
30854,deutschland-tour,2025,stage-2,2025-08-22,Stage 2,p2,Normal,101,Müller Tobias,Intermarché - Wanty,0.0,0.0,2025-08-22 17:15:33
30855,deutschland-tour,2025,stage-2,2025-08-22,Stage 2,p2,Normal,102,Lipowitz Florian,Red Bull - BORA - hansgrohe,0.0,0.0,2025-08-22 17:15:33
30856,deutschland-tour,2025,stage-2,2025-08-22,Stage 2,p2,Normal,103,Zimmermann Georg,Intermarché - Wanty,0.0,0.0,2025-08-22 17:15:33
30857,deutschland-tour,2025,stage-2,2025-08-22,Stage 2,p2,Normal,DNF,Kielich Timo,Alpecin - Deceuninck,0.0,0.0,2025-08-22 17:15:33


In [129]:
%%sql
CREATE TABLE Riders             AS FROM "../data/riders.csv";
CREATE TABLE RidersSelectedBy   AS FROM "../data/riders_selected_by.csv";
CREATE TABLE RiderTeams         AS FROM "../data/rider_teams.csv";
CREATE TABLE Managers           AS FROM "../data/managers.csv";
CREATE TABLE ManagerTeams       AS FROM "../data/manager_teams.csv";
CREATE TABLE ManagerTeamsCheapo AS FROM "../data/manager_cheapo_teams.csv";
CREATE TABLE PointsSystem       AS FROM "../data/points_system.csv";
CREATE TABLE Races              AS FROM "../data/races.csv";
CREATE TABLE Results_2023       AS FROM "../data/results/results_2023_full.csv";
CREATE TABLE Results_2024       AS FROM "../data/results/results_2024_full.csv";
CREATE TABLE Results_2025       AS FROM "../data/results/results_2025_full.csv";
CREATE TABLE CheapoBans         AS FROM "../data/cheapo_bans.csv";

,Count
0,14


In [130]:
%%sql
CREATE OR REPLACE TABLE RaceResultsPoints AS
SELECT
     results.Race
    ,races.RaceName
    ,races.RaceCategory
    ,results.Stage_ID
    ,COALESCE(results.Stage_Date, races.RaceStart) AS Stage_Date
    ,results.Stage_Type
    ,CASE WHEN results.Stage_ID <> 'gc' AND (results.Stage_Type <> 'TTT' OR results.Stage_Type IS NULL) THEN 1 ELSE 0 END AS Løbsdage
    ,results.Rider AS RiderName_PCS
    ,riders.RiderName_Zweeler
    ,riders.RiderPrice
    ,riders.RiderTeam
    ,COALESCE(selected.ValgtAf, 0) AS SelectedBy
    ,CASE WHEN cbans.RiderName IS NULL AND riders.RiderPrice <= 2.5 THEN 'Ja' ELSE '' END AS Cheapo
    ,results.Rank
    ,points.RacePoints AS Points
    ,CASE WHEN races.RaceName = 'Tour Down Under' THEN NULL Else points.RacePoints END AS CheapoPoints
FROM Riders riders
LEFT JOIN Results_2025 results ON results.Rider = riders.RiderName_PCS
LEFT JOIN Races races ON races.RaceName_PCS = results.Race
LEFT JOIN CheapoBans cbans ON cbans.RiderName = riders.RiderName_Zweeler
LEFT JOIN RidersSelectedBy selected ON selected.RiderName_Zweeler = riders.RiderName_Zweeler
LEFT JOIN PointsSystem points ON 1 = 1
    AND (results.Stage_Type IS NULL OR results.Stage_Type <> 'TTT')
    AND points.RaceRank = TRY_CAST(results.Rank AS INTEGER) 
    AND CASE WHEN results.Stage_ID LIKE '%stage%' THEN REPLACE(races.RaceCategory, 'Tour', 'Stage') ELSE races.RaceCategory END = points.RaceCategory;

,Count
0,14527


In [131]:
%%sql
rider_output << 
WITH
ManagerLists AS
(
    SELECT RiderName, STRING_AGG(ManagerName, ', ' ORDER BY ManagerName ASC) AS Managers
    FROM ManagerTeams
    GROUP BY RiderName
),
RiderOverview AS 
(
    SELECT
         RiderName_Zweeler AS Navn
        ,MAX(RiderTeam) AS Hold
        ,MAX(RiderPrice) AS Pris
        ,MAX(SelectedBy) AS Valgt
        ,CAST(SUM(Løbsdage) AS int) AS Løbsdage
        ,COALESCE(CAST(SUM(Points) AS int), 0) AS Point
    FROM RaceResultsPoints
    GROUP BY RiderName_Zweeler
)
SELECT 
     *
    ,ROUND(Point / Pris, 1) AS "P/mil"
    ,ROUND(CASE WHEN Løbsdage = 0 THEN 0 ELSE Point / Løbsdage END, 1) AS "P/dage"
    ,COALESCE((SELECT Managers FROM ManagerLists sub WHERE sub.RiderName = src.Navn), '') AS Managers
FROM RiderOverview src;

In [132]:
%%sql
rider_output_cheapo << 
WITH
ManagerLists AS
(
    SELECT RiderName, STRING_AGG(ManagerName, ', ' ORDER BY ManagerName ASC) AS Managers
    FROM ManagerTeamsCheapo
    GROUP BY RiderName
),
RiderOverview AS 
(
    SELECT
         RiderName_Zweeler AS Navn
        ,MAX(RiderTeam) AS Hold
        ,MAX(RiderPrice) AS Pris
        ,MAX(SelectedBy) AS Valgt
        ,CAST(SUM(Løbsdage) AS int) AS Løbsdage
        ,COALESCE(CAST(SUM(CheapoPoints) AS int), 0) AS Point
        ,MAX(Cheapo) AS Cheapo
    FROM RaceResultsPoints
    GROUP BY RiderName_Zweeler
)
SELECT 
     *
    ,ROUND(Point / Pris, 1) AS "P/mil"
    ,ROUND(CASE WHEN Løbsdage = 0 THEN 0 ELSE Point / Løbsdage END, 1) AS "P/dage"
    ,COALESCE((SELECT Managers FROM ManagerLists sub WHERE sub.RiderName = src.Navn), '') AS Managers
FROM RiderOverview src
WHERE Cheapo = 'Ja';

In [133]:
%%sql
WITH Src AS
(
    SELECT YEARWEEK(results.Stage_Date) AS Uge, teams.ManagerName AS Manager, CAST(SUM(Points) AS int) AS Point
    FROM RaceResultsPoints results
    INNER JOIN ManagerTeams teams ON teams.RiderName = results.RiderName_Zweeler
    WHERE 1 = 1
        AND Uge IS NOT NULL
    GROUP BY YEARWEEK(results.Stage_Date), teams.ManagerName
)
PIVOT Src
ON Uge
USING COALESCE(SUM(Point)::int, 0)
GROUP BY Manager
ORDER BY Manager;

,Manager,202504,202505,202506,202507,202508,202509,202510,202511,202512,202513,202514,202515,202516,202517,202518
0,Chrelle,312,100,67,130,653,72,280,404,125,266,420,572,293,467,58
1,Hustlersen,484,26,6,188,437,0,161,227,127,623,304,258,144,290,89
2,Jappo,156,130,15,130,599,139,379,405,141,547,378,295,207,520,106
3,Jarma,8,105,29,38,557,179,207,284,205,356,394,525,258,339,14
4,Kenk,283,49,106,94,539,110,225,428,167,570,316,464,118,340,110
5,Knak,94,146,15,35,540,96,344,273,153,418,191,400,220,545,52
6,Matti,54,36,214,123,595,63,27,344,118,511,304,355,264,284,48
7,Okholm,183,68,49,22,636,109,162,481,101,361,260,328,200,345,42
8,Tommy,189,94,27,88,414,138,219,269,160,390,454,375,338,322,104
9,Visti,56,106,77,132,425,110,193,378,192,573,311,308,158,379,0


In [134]:
%%sql
dagens_rapport_avanceret <<
WITH Src AS
(
     SELECT 
           CAST(results.Stage_Date AS date) AS Dato
          ,managers.ManagerName AS Manager
          ,CAST(SUM(results.Løbsdage) AS int) AS Løbsdage
          ,SUM(CASE WHEN results.Løbsdage = 1 THEN results.RiderPrice END) AS Millioner
          ,CAST(SUM(results.Points) AS int) AS Point
     FROM RaceResultsPoints results
     LEFT JOIN ManagerTeams managers ON managers.RiderName = results.RiderName_Zweeler
     GROUP BY
           CAST(results.Stage_Date AS DATE)
          ,managers.ManagerName
), ManagersDatoer AS
(
     SELECT 
           Manager
          ,Dato FROM
     (SELECT DISTINCT Manager FROM Src) m
     CROSS JOIN (SELECT DISTINCT Dato FROM Src) d
     WHERE m.Manager IS NOT NULL AND d.Dato IS NOT NULL
)
SELECT
      managersdatoer.Manager
     ,managersdatoer.Dato
     ,COALESCE(summer.Løbsdage, 0) AS Løbsdage
     ,COALESCE(summer.Millioner, 0) AS Millioner
     ,COALESCE(summer.Point, 0) AS Point
     ,ROUND(CASE WHEN COALESCE(summer.Løbsdage, 0) = 0 THEN 0 ELSE COALESCE(summer.Point, 0) / COALESCE(summer.Løbsdage, 0) END, 2) AS "P/rytter"
     ,ROUND(CASE WHEN COALESCE(summer.Millioner, 0) = 0 THEN 0 ELSE COALESCE(summer.Point, 0) / COALESCE(summer.Millioner, 0) END, 2) AS "P/mil"
     ,(
          SELECT COALESCE(STRING_AGG(RiderPoints, ', ' ORDER BY Points DESC), '')
          FROM
               (   SELECT
                         CONCAT(
                              SUBSTRING(r.RiderName_Zweeler, 1, position(', ' IN r.RiderName_Zweeler) - 1), 
                              ' (',
                              CAST(SUM(r.Points) AS int), 
                              ')'
                         ) AS RiderPoints,
                         SUM(r.Points) AS Points
                    FROM RaceResultsPoints r
                    INNER JOIN ManagerTeams m ON m.RiderName = r.RiderName_Zweeler
                    WHERE 1 = 1
                         AND CAST(r.Stage_Date AS date) = managersdatoer.Dato
                         AND m.ManagerName = managersdatoer.Manager
                         AND r.Points IS NOT NULL
                    GROUP BY r.RiderName_Zweeler
                    ORDER BY SUM(r.Points) DESC
                    LIMIT 3
               ) t
      ) AS "Top 3 ryttere"
FROM ManagersDatoer managersdatoer
LEFT JOIN Src summer ON managersdatoer.Manager = summer.Manager AND managersdatoer.Dato = summer.Dato
ORDER BY 
      managersdatoer.Dato DESC
     ,COALESCE(summer.Point, 0) DESC;

In [135]:
%%sql
ugens_rapport_avanceret <<
WITH Src AS
(
     SELECT 
           YEARWEEK(CAST(results.Stage_Date AS date)) AS Uge
          ,managers.ManagerName AS Manager
          ,CAST(SUM(results.Løbsdage) AS int) AS Løbsdage
          ,SUM(CASE WHEN results.Løbsdage = 1 THEN results.RiderPrice END) AS Millioner
          ,CAST(SUM(results.Points) AS int) AS Point
     FROM RaceResultsPoints results
     LEFT JOIN ManagerTeams managers ON managers.RiderName = results.RiderName_Zweeler
     GROUP BY
           YEARWEEK(CAST(results.Stage_Date AS date))
          ,managers.ManagerName
), ManagersDatoer AS
(
     SELECT 
           Manager
          ,Uge FROM
     (SELECT DISTINCT Manager FROM Src) m
     CROSS JOIN (SELECT DISTINCT Uge FROM Src) d
     WHERE m.Manager IS NOT NULL AND d.Uge IS NOT NULL
)
SELECT
      managersdatoer.Manager
     ,managersdatoer.Uge
     ,COALESCE(summer.Løbsdage, 0) AS Løbsdage
     ,COALESCE(summer.Millioner, 0) AS Millioner
     ,COALESCE(summer.Point, 0) AS Point
     ,ROUND(CASE WHEN COALESCE(summer.Løbsdage, 0) = 0 THEN 0 ELSE COALESCE(summer.Point, 0) / COALESCE(summer.Løbsdage, 0) END, 2) AS "P/rytter"
     ,ROUND(CASE WHEN COALESCE(summer.Millioner, 0) = 0 THEN 0 ELSE COALESCE(summer.Point, 0) / COALESCE(summer.Millioner, 0) END, 2) AS "P/mil"
     ,(
          SELECT COALESCE(STRING_AGG(RiderPoints, ', ' ORDER BY Points DESC), '')
          FROM
               (   SELECT
                         CONCAT(
                              SUBSTRING(r.RiderName_Zweeler, 1, position(', ' IN r.RiderName_Zweeler) - 1), 
                              ' (',
                              CAST(SUM(r.Points) AS int), 
                              ')'
                         ) AS RiderPoints,
                         SUM(r.Points) AS Points
                    FROM RaceResultsPoints r
                    INNER JOIN ManagerTeams m ON m.RiderName = r.RiderName_Zweeler
                    WHERE 1 = 1
                         AND YEARWEEK(CAST(r.Stage_Date AS date)) = managersdatoer.Uge
                         AND m.ManagerName = managersdatoer.Manager
                         AND r.Points IS NOT NULL
                    GROUP BY r.RiderName_Zweeler
                    ORDER BY SUM(r.Points) DESC
                    LIMIT 3
               ) t
      ) AS "Top 3 ryttere"
FROM ManagersDatoer managersdatoer
LEFT JOIN Src summer ON managersdatoer.Manager = summer.Manager AND managersdatoer.Uge = summer.Uge
ORDER BY 
      managersdatoer.Uge DESC
     ,COALESCE(summer.Point, 0) DESC;

In [136]:
rider_output.to_html("../outputtables/RytteroversigtTabel.html", table_id = "filterabletable", index = False)
rider_output_cheapo.to_html("../outputtables/RytteroversigtCheapoTabel.html", table_id = "filterabletable", index = False)
dagens_rapport_avanceret.to_html("../outputtables/DagensRapportAvanceret.html", table_id = "filterabletable", index = False)
ugens_rapport_avanceret.to_html("../outputtables/UgensRapportAvanceret.html", table_id = "filterabletable", index = False)

In [157]:
%%sql
WITH Src AS
(
    SELECT 
        managers.ManagerName AS Manager
        ,YEAR(results.Stage_Date) * 100 + MONTH(results.Stage_Date) AS ÅrMåned
        ,SUM(results.Løbsdage * results.RiderPrice) AS SamletPris
        ,SUM(results.Løbsdage) AS SamledeLøbsdage
        ,SUM(results.Points) AS SamledePoint
    FROM RaceResultsPoints results
    LEFT JOIN ManagerTeams managers ON managers.RiderName = results.RiderName_Zweeler
    WHERE 1 = 1
        AND managers.ManagerName IS NOT NULL
    GROUP BY
        managers.ManagerName
        ,YEAR(results.Stage_Date) * 100 + MONTH(results.Stage_Date)
), Managers AS
(
    SELECT DISTINCT Manager
    FROM Src
)
SELECT 
     m.Manager
    ,ROUND(COALESCE((SELECT SamledePoint / SamletPris FROM Src s WHERE s.Manager = m.Manager AND s.ÅrMåned = 202501), 0), 2) AS "202501"
    ,ROUND(COALESCE((SELECT SamledePoint / SamletPris FROM Src s WHERE s.Manager = m.Manager AND s.ÅrMåned = 202502), 0), 2) AS "202502"
    ,ROUND(COALESCE((SELECT SamledePoint / SamletPris FROM Src s WHERE s.Manager = m.Manager AND s.ÅrMåned = 202503), 0), 2) AS "202503"
    ,ROUND(COALESCE((SELECT SamledePoint / SamletPris FROM Src s WHERE s.Manager = m.Manager AND s.ÅrMåned = 202504), 0), 2) AS "202504"
    ,ROUND(COALESCE((SELECT SamledePoint / SamletPris FROM Src s WHERE s.Manager = m.Manager AND s.ÅrMåned = 202505), 0), 2) AS "202505"
    ,ROUND(COALESCE((SELECT SamledePoint / SamletPris FROM Src s WHERE s.Manager = m.Manager AND s.ÅrMåned = 202506), 0), 2) AS "202506"
    ,ROUND(COALESCE((SELECT SamledePoint / SamletPris FROM Src s WHERE s.Manager = m.Manager AND s.ÅrMåned = 202507), 0), 2) AS "202507"
    ,ROUND(COALESCE((SELECT SUM(SamletPris) FROM Src s WHERE s.Manager = m.Manager), 0), 2) AS "Millioner i alt"
    ,CAST(ROUND(COALESCE((SELECT SUM(SamledePoint) FROM Src s WHERE s.Manager = m.Manager), 0), 2) AS int) AS "Point i alt"
    ,ROUND(COALESCE((SELECT SUM(SamledePoint) / SUM(SamletPris) FROM Src s WHERE s.Manager = m.Manager), 0), 2) AS "Total"
FROM Managers m;

,Manager,202501,202502,202503,202504,202505,202506,202507,Millioner i alt,Point i alt,Total
0,Visti,0.44,0.39,0.62,1.18,0.00,0.0,0.0,5398.3,3398,0.63
1,Chrelle,1.30,0.49,0.61,1.22,0.45,0.0,0.0,5563.4,4219,0.76
2,Tommy,0.70,0.39,0.57,1.18,0.96,0.0,0.0,5281.3,3581,0.68
3,Knak,0.51,0.42,0.58,1.10,0.62,0.0,0.0,5446.3,3522,0.65
4,Jarma,0.07,0.42,0.59,1.34,0.00,0.0,0.0,5158.1,3498,0.68
5,Hustlersen,1.40,0.47,0.60,1.07,0.68,0.0,0.0,4673.6,3364,0.72
6,Okholm,0.91,0.42,0.58,0.91,0.79,0.0,0.0,5454.3,3347,0.61
7,Kenk,1.45,0.44,0.65,0.99,0.84,0.0,0.0,5689.5,3919,0.69
8,Matti,0.76,0.47,0.50,0.86,0.45,0.0,0.0,5739.3,3340,0.58
9,Jappo,0.83,0.48,0.67,1.07,1.16,0.0,0.0,5747.9,4147,0.72
